In [93]:
import pandas as pd
import os
df_path = r"C:\MSAI\Cases in ML\Final Project\Datasets"

df = pd.read_csv(os.path.join(df_path, 'TMDB_all_movies.csv'), low_memory=False)
df = df[(~df['title'].isna()) & (df['status'] == 'Released') & (~df['release_date'].isna()) & (df['original_language'] == 'en')]
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year
df['release_year'] = df['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df['title_processed'] = df['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
columns = [
    'id',
    'imdb_id',
    'title',
    'title_processed',
    'original_title',
    'release_date',
    'release_year',
    'overview',
    'revenue', 
    'budget',
    'runtime', 
    'popularity',
    'tagline',
    'genres',
    'production_companies',
    'production_countries',
    'cast',
    'director',
    'writers',
    'producers',
    'imdb_rating',
    'imdb_votes',
]
df = df[columns]
df.fillna({'budget': 0, 'revenue': 0}, inplace=True)

#get data from 'imdb_movies.csv'
df1 = pd.read_csv(os.path.join(df_path, 'imdb_movies.csv'), low_memory=False)
df1.rename(columns={'orig_title': 'title', 'date_x': 'release_date', 'revenue': 'revenue_1', 'budget_x': 'budget_1', 'crew': 'cast_1'}, inplace=True)
df1 = df1[(~df1['title'].isna()) & (~df1['release_date'].isna())]
df1 = df1[df1['status'].str.contains('Released')]
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce')
df1['release_year'] = df1['release_date'].dt.year
df1['imdb_score_1'] = df1['score'] * 0.1
df1['title_processed'] = df1['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df1['release_year'] = df1['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df1.fillna({'budget_1': 0, 'revenue_1': 0}, inplace=True)
df1 = df1[['title_processed', 'release_year', 'budget_1', 'revenue_1', 'imdb_score_1', 'cast_1']].drop_duplicates()
df = pd.merge(df, df1, on=['title_processed', 'release_year'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_1']), axis=1)
df['budget'] = df.apply(lambda row: max(row['budget'], row['budget_1']), axis=1)
df['imdb_rating'] = df.apply(lambda row: row['imdb_rating'] if row['imdb_rating'] else row['imdb_rating_1'], axis=1)
df['cast'] = df.apply(lambda row: row['cast'] if row['cast'] else row['cast_1'], axis=1)
df.drop(columns=['budget_1', 'revenue_1', 'imdb_score_1', 'cast_1'], inplace=True)

# #get data from 'movies.csv'
df2 = pd.read_csv("https://raw.githubusercontent.com/danielgrijalva/movie-stats/master/movies.csv", low_memory=False)
df2.rename(columns={'name': 'title', 'year': 'release_year', 'gross': 'revenue_2', 'budget': 'budget_2'}, inplace=True)
df2 = df2[(~df2['title'].isna()) & (~df2['release_year'].isna())]
df2['title_processed'] = df2['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df2['release_year'] = df2['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df2.fillna({'revenue_2': 0, 'budget_2': 0}, inplace=True)
df2 = df2[['title_processed', 'release_year', 'budget_2', 'revenue_2']].drop_duplicates()
df = pd.merge(df, df2, on=['title_processed', 'release_year'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_2']), axis=1)
df['budget'] = df.apply(lambda row: max(row['budget'], row['budget_2']), axis=1)
df.drop(columns=['budget_2', 'revenue_2'], inplace=True)

# #get data from 'top_movies_data.csv'
df3 = pd.read_csv(os.path.join(df_path, 'top_movies_data.csv'), low_memory=False)
df3.rename(columns={'production_year': 'release_year', 'budget': 'budget_3'}, inplace=True)
df3 = df3[(~df3['title'].isna()) & (~df3['release_year'].isna())]
df3['title_processed'] = df3['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df3['release_year'] = df3['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df3.fillna({'budget_3': 0}, inplace=True)
df3 = df3[['title_processed', 'release_year', 'budget_3']].drop_duplicates()
df = pd.merge(df, df3, on=['title_processed', 'release_year'], how='left')
df['budget'] = df.apply(lambda row: max(row['budget'], row['budget_3']), axis=1)
df.drop(columns=['budget_3'], inplace=True)

#get data from 'Details_budgets.csv'
df4 = pd.read_csv(os.path.join(df_path, 'Details_budgets.csv'), low_memory=False)
df4.rename(columns={'Movie': 'title', 'Release Year': 'release_year'}, inplace=True)
df4 = df4[(~df4['title'].isna()) & (~df4['release_year'].isna())]
df4['budget_4'] = df4['Budget ($M)'] * 1000000
df4['revenue_4'] = df4['Worldwide ($M)'] * 1000000
df4['title_processed'] = df4['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df4['release_year'] = df4['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df4.fillna({'revenue_4': 0, 'budget_4': 0}, inplace=True)
df4 = df4[['title_processed', 'release_year', 'budget_4', 'revenue_4']].drop_duplicates()
df = pd.merge(df, df4, on=['title_processed', 'release_year'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_4']), axis=1)
df['budget'] = df.apply(lambda row: max(row['budget'], row['budget_4']), axis=1)
df.drop(columns=['budget_4', 'revenue_4'], inplace=True)

#get data from 'the-numbers.csv'
df5 = pd.read_csv(os.path.join(df_path, 'the-numbers.csv'), low_memory=False)
df5.rename(columns={'Movie': 'title', 'Worldwide Gross': 'revenue_5', 'Production Budget': 'budget_5'}, inplace=True)
df5['release_date'] = pd.to_datetime(df5['Release Date'], errors='coerce')
df5 = df5[(~df5['title'].isna()) & (~df5['release_year'].isna())]
df5['release_year'] = df5['release_date'].dt.year
df5['title_processed'] = df5['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df5['release_year'] = df5['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df5.fillna({'revenue_5': 0, 'budget_5': 0}, inplace=True)
df5 = df5[['title_processed', 'release_year', 'revenue_5', 'budget_5']].drop_duplicates()
df = pd.merge(df, df5, on=['title_processed', 'release_year'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_5']), axis=1)
df['budget'] = df.apply(lambda row: max(row['budget'], row['budget_5']), axis=1)
df.drop(columns=['budget_5', 'revenue_5'], inplace=True)

#get data from 'IMDb_All_Genres_etf_clean1.csv'
df6 = pd.read_csv(os.path.join(df_path, 'IMDb_All_Genres_etf_clean1.csv'), low_memory=False)
df6 = df6[(~df6['Total_Gross'].isna()) & (df6['Total_Gross'] != 'Gross Unkown')]
df6.rename(columns={'Movie_Title': 'title', 'Year': 'release_year'}, inplace=True)
df6['title_processed'] = df6['title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df6['release_year'] = df6['release_year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df6['Total_Gross'] = df6['Total_Gross'].str.replace(r'\D', '', regex=True).astype(int)
df6['revenue_6'] = df6['Total_Gross'].astype(int) * 10000
df6.fillna({'revenue_6': 0}, inplace=True)
df6 = df6[['title_processed', 'release_year', 'revenue_6']].drop_duplicates()
df = pd.merge(df, df6, on=['title_processed', 'release_year'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_6']), axis=1)
df.drop(columns=['revenue_6'], inplace=True)

#get data from 'imdb_data.csv'
df7 = pd.read_csv(os.path.join(df_path, 'imdb_data.csv'), low_memory=False)
df7.rename(columns={'id': 'imdb_id', 'gross': 'revenue_7', 'budget': 'budget_7', 'averageRating': 'imdb_rating_7', 'directors': 'director_7'}, inplace=True)
df7.fillna({'revenue_7': 0, 'budget_7': 0}, inplace=True)
df7 = df7[['imdb_id', 'revenue_7', 'budget_7', 'imdb_rating_7', 'director_7']].drop_duplicates()
df = pd.merge(df, df7, on=['imdb_id'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_7']), axis=1)
df['budget'] = df.apply(lambda row: max(row['budget'], row['budget_7']), axis=1)
df['imdb_rating'] = df.apply(lambda row: row['imdb_rating'] if row['imdb_rating'] else row['imdb_rating_7'], axis=1)
df['director'] = df.apply(lambda row: row['director'] if row['director'] else row['director_7'], axis=1)
df.drop(columns=['revenue_7', 'budget_7', 'imdb_rating_7', 'director_7'], inplace=True)

#get data from 'IMDB-Movie-Data.csv'
df8 = pd.read_csv(os.path.join(df_path, 'IMDB-Movie-Data.csv'), low_memory=False)
df8 = df8[(~df8['Year'].isna()) & (df8['Title'].isna()) & (~df8['Revenue (Millions)'].isna())]
df8['release_year'] = df8['Year'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df8['title_processed'] = df8['Title'].str.lower().str.replace('[^a-z0-9 ]', '', regex=True)
df8['Revenue (Millions)'] = df8['Revenue (Millions)'].astype(int)
df8['revenue_8'] = df8['Revenue (Millions)'].astype(int) * 1000000
df8.fillna({'revenue_8': 0}, inplace=True)
df8 = df8[['title_processed', 'release_year', 'revenue_8']].drop_duplicates()
df = pd.merge(df, df8, on=['title_processed', 'release_year'], how='left')
df['revenue'] = df.apply(lambda row: max(row['revenue'], row['revenue_8']), axis=1)
df.drop(columns=['revenue_8'], inplace=True)

df.drop_duplicates(subset=['imdb_id', 'title_processed', 'release_year'], inplace=True)

df.to_csv(os.path.join(df_path, 'final_agg.csv'), index=False)

In [1]:
import pandas as pd
import os

df_path = r"C:\MSAI\Cases in ML\Final Project\Datasets"

df = pd.read_csv(os.path.join(df_path, 'final_agg.csv'), low_memory=False)
df9 = pd.read_csv(os.path.join(df_path, 'budget.csv'), low_memory=False)
df9.rename(columns={'revenue': 'revenue_9', 'budget': 'budget_9'}, inplace=True)
df99 = pd.merge(df, df9, on=['imdb_id'], how='left')
df99['revenue'] = df99.apply(lambda row: max(row['revenue'], row['revenue_9']), axis=1)
df99['budget'] = df99.apply(lambda row: max(row['budget'], row['budget_9']), axis=1)
df99.drop(columns=['budget_9', 'revenue_9'], inplace=True)

df99.to_csv(os.path.join(df_path, 'final.csv'), index=False)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from random import randint
from time import sleep

def get_soup_from_url(url):
    headers = {'Accept-Language': 'en',
               'X-FORWARDED_FOR': f"{randint(1,200)}.{randint(1,200)}.{randint(1,200)}.{randint(1,200)}"}
    response = requests.get(url, headers=headers)
    assert response.status_code == 200, response.status_code
    soup = BeautifulSoup(response.text, features="html.parser")
    return soup

def get_movie_budgets(url):
    table = get_soup_from_url(url).find('table')
    ids = []
    release_dates = []
    movies = []
    budgets = []
    domestic_gross = []
    worldwide_gross = []
    
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) >= 6:
            ids.append(cols[0].text.strip())
            release_dates.append(cols[1].text.strip())
            movies.append(cols[2].text.strip())
            budgets.append(cols[3].text.strip())
            domestic_gross.append(cols[4].text.strip())
            worldwide_gross.append(cols[5].text.strip())
    
    df = pd.DataFrame({
        'ID': ids,
        'Release Date': release_dates,
        'Movie': movies,
        'Production Budget': budgets,
        'Domestic Gross': domestic_gross,
        'Worldwide Gross': worldwide_gross
    })
    
    df['Production Budget'] = df['Production Budget'].str.replace(r'\D', '', regex=True)
    df['Domestic Gross'] = df['Domestic Gross'].str.replace(r'\D', '', regex=True)
    df['Worldwide Gross'] = df['Worldwide Gross'].str.replace(r'\D', '', regex=True)
    
    return df

def get_all_movie_budgets():
    base_url = 'https://www.the-numbers.com/movie/budgets/all'
    movie_data = get_movie_budgets(url=base_url)
    page = 1
    
    while True:
        try:
            page += 100
            next_url = f"{base_url}/{page}"
            df = get_movie_budgets(url=next_url)
            
            if df.empty or len(df) < 99:  # Last page?
                break
                
            movie_data = pd.concat([movie_data, df], ignore_index=True)
            sleep(5)
            print(f"Current size of movies: {len(movie_data)}...")
            
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break
    
    return movie_data

# Get the data
movie_data = get_all_movie_budgets()   
movie_data

In [ ]:
# df99 = pd.read_csv(os.path.join(df_path, 'final.csv'), low_memory=False)
missing_df = df99[((df99['revenue'] == 0) | (df99['budget'] == 0)) & (~df99['imdb_id'].isna())]['imdb_id']

TMDB_API = "...." # Here The Movie Database API key

def get_movie_info_by_imdb_id(imdb_id, tmdb_api):
    base_url = "https://api.themoviedb.org/3/find/{}"
    params = {
        'api_key': tmdb_api,
        'external_source': 'imdb_id'
    }
    response = requests.get(base_url.format(imdb_id), params=params)
    assert response.status_code == 200, response.status_code
    tmdb_id = response.json().get("movie_results", [])[0].get('id')
    url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?api_key={tmdb_api}"
    resp = requests.get(url)
    assert resp.status_code == 200, resp.status_code
    budget = resp.json().get("budget", 0)
    revenue = resp.json().get("revenue", 0)
    return {'budget': budget, 'revenue': revenue}

def get_soup_from_url(url):
    headers = {'Accept-Language': 'en', #'X-FORWARDED_FOR': '1.12.1.12'}
               'X-FORWARDED_FOR': f"{randint(1,200)}.{randint(1,200)}.{randint(1,200)}.{randint(1,200)}"}
    response = requests.get(url, headers=headers)
    assert response.status_code == 200, response.status_code
    soup = BeautifulSoup(response.text, features="html.parser")
    return soup

def get_budget(imdb_id):
    box_office_soup = get_soup_from_url(f"https://www.boxofficemojo.com/title/{imdb_id}")
    box_office_data = box_office_soup.find('div', attrs={'class': 'mojo-performance-summary-table'}).find_all('span', attrs={'class': 'money'})
    try:
        worldwide = int(''.join(filter(str.isdigit, box_office_data[-1].get_text())))
    except:
        worldwide = 0
    try:
        budget = int(''.join(
            filter(str.isdigit, 
                   box_office_soup.find('span', string='Budget').find_next_sibling('span').find('span', class_='money').get_text())
                )
            )
    except:
        budget = 0

    return {'budget': budget, 'revenue': worldwide}

data = defaultdict(dict)
df = pd.read_csv(os.path.join(df_path, 'budget.csv'))
if len(df) > 0:
    for _, row in df.iterrows():
        data[row['imdb_id']] = {'budget': row['budget'], 'revenue': row['revenue']}

print(f"Total number of movies with missing details: {len(missing_df)}")

for item in list(missing_df):
    if item not in data:
        try:
            data[item] = get_budget(item)
            try:
                if data[item]['budget'] == 0 or data[item]['revenue'] == 0:
                    details = get_movie_info_by_imdb_id(imdb_id=item, tmdb_api=TMDB_API)
                    data[item] = {
                        'budget': max(data[item]['budget'], details['budget']),
                        'revenue': max(data[item]['revenue'], details['revenue'])
                    }
            except:
                pass
            print(f"Current size of movies: {len(data)}")
            sleep(3)  # Delay between requests (avoid blocking by rate limiter)
            
            
        except Exception as e:
            print(f"Error fetching page {item}: {e}")
            break
            
        df = pd.DataFrame([
            {'imdb_id': imdb_id, 'budget': values['budget'], 'revenue': values['revenue']}
            for imdb_id, values in data.items()
        ])
        df.to_csv(os.path.join(df_path, 'budget.csv'), index=False)

print("DONE!")

Total number of movies with missing details: 267910
Current size of movies: 28654
Current size of movies: 28655
Current size of movies: 28656
Current size of movies: 28657
Current size of movies: 28658
Current size of movies: 28659
Current size of movies: 28660
Current size of movies: 28661
Current size of movies: 28662
Current size of movies: 28663
Current size of movies: 28664
Current size of movies: 28665
Current size of movies: 28666
Current size of movies: 28667
Current size of movies: 28668
Current size of movies: 28669
Current size of movies: 28670
Current size of movies: 28671
Current size of movies: 28672
Current size of movies: 28673
Current size of movies: 28674
Current size of movies: 28675
Current size of movies: 28676
Current size of movies: 28677
Current size of movies: 28678
Current size of movies: 28679
Current size of movies: 28680
Current size of movies: 28681
Current size of movies: 28682
Current size of movies: 28683
Current size of movies: 28684
Current size of mo